In [5]:
import pandas as pd
import numpy as np
import re

In [6]:
mask = pd.read_csv('./data/mask_text.csv')
vaccine = pd.read_csv('./data/vaccine_text.csv')

C:\Users\wongc\anaconda3\envs\comp4211\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\wongc\anaconda3\envs\comp4211\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,3,7,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
vaccine

,date,user,is_retweet,is_quote,text,quoted_text,lat,long,hts,mentions,tweet_id,likes,retweets,replies,quote_count,original_tweet_id
0,"being of the citizens of the planet"". #CarlSag...",NaN,NaN,NaN,USElections2020;USA;USElection2020;COVID19;cor...,NaN,1.3233829940993188e+18,0.0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
1,2020-11-02 22:00:39,0Blair0,True,True,RT @JackPosobiec: The WHO helped China cover u...,China extracted concessions from the WHO that ...,NaN,NaN,NaN,NaN,1.3233845543678894e+18,0.0,0.0,0.0,0.0,1.323384e+18
2,2020-11-02 20:46:16,0CT0BERSURPRISE,True,False,RT @JoeBiden: This is my commitment to you: We...,NaN,NaN,NaN,NaN,NaN,1.323365833528103e+18,0.0,0.0,0.0,0.0,1.323291e+18
3,2020-11-03 13:26:58,0Gf1lCKkKy0ucqy,True,False,RT @HHShkMohd: While receiving the COVID-19 va...,NaN,NaN,NaN,NaN,NaN,1323617667178205186,0,0.0,0.0,0.0,1.323543e+18
4,2020-11-03 07:44:24,0Selna0,True,True,RT @angie_rasmussen: The most likely scenario ...,This long New York Times story on China’s refu...,NaN,NaN,NaN,NaN,1323531459957719042,0,0.0,0.0,0.0,1.323448e+18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69931,2020-11-03 10:39:58,zyyyye,True,False,"RT @PhilstarNews: Instead of a medical expert,...",NaN,NaN,NaN,NaN,NaN,1323575641959096320.0,0.0,0.0,0.0,0.0,1.323435e+18
69932,2020-11-03 07:00:06,zzposts,False,False,Frozen Food Packages in China Keep Testing Pos...,NaN,NaN,NaN,NaN,NaN,1323520309480271872.0,0.0,0.0,0.0,0.0,NaN
69933,2020-11-03 10:00:05,zzposts,False,False,UAE PM receives Covid-19 vaccine shot https://...,NaN,NaN,NaN,health;read;proud;covid19;rashid;safety,NaN,1323565603530199040.0,0.0,0.0,0.0,0.0,NaN
69934,2020-11-02 23:47:56,zzzdeu,True,True,RT @JackPosobiec: The WHO helped China cover u...,China extracted concessions from the WHO that ...,NaN,NaN,NaN,NaN,1323411550397497088.0,0.0,0.0,0.0,0.0,1.323384e+18


In [8]:
def remove_rt(string):
    return re.sub(r"^RT @\w ", "", string)

# From "@musicmadmarc @SocialDilemma_ @netflix @Facebook I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"
# to: "I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"

def remove_beginning_mentions(string):
#     print(string)
#     if(re.search(r'@\w+',string) != None):
    # if(string =='@'):
    new_string=re.sub(r'^[@\w ]*','',string,1)
    new_string = new_string.strip()
    string = remove_beginning_mentions(new_string)
    return string

# From "#musicmadmarc #SocialDilemma_ #netflix #Facebook I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"
# to: "I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"

def remove_beginning_hashs(string):
    # if(string[0]=='#'):
    new_string=re.sub(r'^[#\w ]*','',string,1)
    new_string = new_string.strip()
    string = remove_beginning_mentions(new_string)
    return string

# From " I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K @musicmadmarc @SocialDilemma_ @netflix @Facebook"
# to: "I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"

def remove_last_mentions(string):
    last_mention = string.rfind('@')
    last_space = string.rfind(' ')
    if(last_mention > last_space):
        string = string[:last_space]
        string = remove_last_mentions(string)
    return string

# From " I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K @musicmadmarc @SocialDilemma_ @netflix @Facebook"
# to: "I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"

def remove_last_hashs(string):
    last_mention = string.rfind('#')
    last_space = string.rfind(' ')
    if(last_mention > last_space):
        string = string[:last_space]
        string = remove_last_mentions(string)
    return string

def remove_punctuation_notation(string):
    string = re.sub(r'[\$%&\'()*+\-\/=#@\[\\\]^_`{|}~]*','',string)
    return string

def preprocessing_pipeline(string):
    string = remove_rt(string)
    string = remove_beginning_mentions(string)
    string = remove_beginning_hashs(string)
    string = remove_last_mentions(string)
    string = remove_last_hashs(string)
    string = remove_punctuation_notation(string)
    return string


In [9]:
vaccine.text = vaccine.text.apply(lambda x:preprocessing_pipeline(x))

In [8]:
vaccine   

,date,user,is_retweet,is_quote,text,quoted_text,lat,long,hts,mentions,tweet_id,likes,retweets,replies,quote_count,original_tweet_id
0,"being of the citizens of the planet"". #CarlSag...",NaN,NaN,NaN,USElections2020;USA;USElection2020;COVID19;cor...,NaN,1.3233829940993188e+18,0.0,0,0,0.0,NaN,NaN,NaN,NaN,NaN
1,2020-11-02 22:00:39,0Blair0,True,True,ackPosobiec: The WHO helped China cover up COVID,China extracted concessions from the WHO that ...,NaN,NaN,NaN,NaN,1.3233845543678894e+18,0.0,0.0,0.0,0.0,1.323384e+18
2,2020-11-02 20:46:16,0CT0BERSURPRISE,True,False,oeBiden: This is my commitment to you: We will...,NaN,NaN,NaN,NaN,NaN,1.323365833528103e+18,0.0,0.0,0.0,0.0,1.323291e+18
3,2020-11-03 13:26:58,0Gf1lCKkKy0ucqy,True,False,HShkMohd: While receiving the COVID-19 vaccine...,NaN,NaN,NaN,NaN,NaN,1323617667178205186,0,0.0,0.0,0.0,1.323543e+18
4,2020-11-03 07:44:24,0Selna0,True,True,ngie_rasmussen: The most likely scenario is na...,This long New York Times story on China’s refu...,NaN,NaN,NaN,NaN,1323531459957719042,0,0.0,0.0,0.0,1.323448e+18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69931,2020-11-03 10:39:58,zyyyye,True,False,"hilstarNews: Instead of a medical expert, Pres...",NaN,NaN,NaN,NaN,NaN,1323575641959096320.0,0.0,0.0,0.0,0.0,1.323435e+18
69932,2020-11-03 07:00:06,zzposts,False,False,Frozen Food Packages in China Keep Testing Pos...,NaN,NaN,NaN,NaN,NaN,1323520309480271872.0,0.0,0.0,0.0,0.0,NaN
69933,2020-11-03 10:00:05,zzposts,False,False,UAE PM receives Covid-19 vaccine shot https://...,NaN,NaN,NaN,health;read;proud;covid19;rashid;safety,NaN,1323565603530199040.0,0.0,0.0,0.0,0.0,NaN
69934,2020-11-02 23:47:56,zzzdeu,True,True,ackPosobiec: The WHO helped China cover up COVID,China extracted concessions from the WHO that ...,NaN,NaN,NaN,NaN,1323411550397497088.0,0.0,0.0,0.0,0.0,1.323384e+18


In [ ]:
vaccine.to_csv('./data/vaccine_text.csv',index = False)